In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
load_dotenv()

True

In [3]:
from crewai import LLM
model = LLM(
    model="google/gemma-3n-e2b-it:free",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0.7,
)

In [22]:
original_email = """hey ayush can u pls chng the monday OOPJ n AWT class time frm 12-2 to 3-5 its free slot i hv to go home for sum emergncy wrk. dipanshu bca eve 24-27"""


Simple Tool execution


In [26]:
from crewai.tools import BaseTool

class ReplaceJargonsTools(BaseTool):
   name:str = "Replace Jargons"
   description:str = "Replace Jargons with more specific terms"

   def _run(self,email:str) ->str:
       replacements={
           "OOPJ": "Java",
           "AWT" : "Advanced web technology",
           "EVS" : "Environmental science",
           "BCA": "Bachelor of Computer Applications",
           "u": "you",
           "pls": "please",
       }

       suggestions = []
       email_lower = email.lower()
       for jargon,replacement in replacements.items():
           if jargon.lower() in email_lower:
               suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")
       if not suggestions:
           return "No jargon or abbreviations detected that need replacement."

       return "\n".join(suggestions)
   
jargon_tool = ReplaceJargonsTools()
jargon_tool.run(original_email)

Using Tool: Replace Jargons


"Consider replacing 'OOPJ' with 'Java'\nConsider replacing 'AWT' with 'Advanced web technology'\nConsider replacing 'BCA' with 'Bachelor of Computer Applications'\nConsider replacing 'u' with 'you'\nConsider replacing 'pls' with 'please'"

In [27]:
from crewai import Agent,Task,Crew

email_assistant = Agent(
    role="Email Assistant Agent",
    goal="Improve emails and make them sound professional and clear",
    backstory="A highly experienced communication expert skilled in professional email writing",
    verbose=True,
    tools=[jargon_tool],
    llm = model
)

In [28]:
description = "You are an expert communication assistant. Improve the following email to make it more clear, polite, and professional while keeping the original intent unchanged. Correct grammar and improve sentence structure as needed."

In [29]:
email_task = Task(
    description=f"""{description} 
     Original email to improve:
    '''{original_email}'''""",
    agent = email_assistant,
    expected_output="A professional written email with proper formatting and content"
)

In [32]:
crew = Crew(
    agents = [email_assistant],
    tasks = [email_task],
    verbose=False
)


import sys
result = crew.kickoff()
sys.stdout.flush()  # Force output to display
print(f"\n\nFINAL RESULT:\n{result}")


┌───────────────────────────── 🤖 Agent Started ──────────────────────────────┐
│                                                                             │
│  Agent: Email Assistant Agent                                               │
│                                                                             │
│  Task: You are an expert communication assistant. Improve the following     │
│  email to make it more clear, polite, and professional while keeping the    │
│  original intent unchanged. Correct grammar and improve sentence structure  │
│  as needed.                                                                 │
│       Original email to improve:                                            │
│      '''hey ayush can u pls chng the monday OOPJ n AWT class time frm 12-2  │
│  to 3-5 its free slot i hv to go home for sum emergncy wrk. dipanshu bca    │
│  eve 24-27'''                                                               │
│                                        

┌────────────────────────── 🔧 Agent Tool Execution ──────────────────────────┐
│                                                                             │
│  Agent: Email Assistant Agent                                               │
│                                                                             │
│  Thought: First, let's clarify the original message for better              │
│  understanding:                                                             │
│  "Hey Ayush, can you please change the Monday OOPJ and AWT class time from  │
│  12-2 to 3-5. It's a free slot, I have to go home for some emergency work.  │
│  Dipanshu (BCA) Eve 24-27."                                                 │
│                                                                             │
│  Using Tool: Replace Jargons                                                │
│                                                                             │
└────────────────────────────────────────

In [33]:
with open("improved_email.txt", "w", encoding="utf-8") as f:
    f.write("ORIGINAL EMAIL:\n")
    f.write(original_email)
    f.write("\n\nIMPROVED EMAIL:\n")
    f.write(str(result))

print("Result also saved to 'improved_email.txt'")

Result also saved to 'improved_email.txt'
